In [1]:
from pandas import read_csv, to_datetime, DataFrame, Series, concat, merge
from datetime import timedelta

In [2]:
positions = read_csv('data/positions.csv').fillna(0.0)
securities = list(set(positions['Security']))

In [3]:
ticker_name = read_csv('data/SF0-tickers.csv', index_col=0)
tickers = [ticker_name.loc[ticker_name['Name'] == sec]['Ticker'].values[0] for sec in securities]

In [4]:
positions.head()

,Date,Security,Predicted Z-score for transformed percent change,Strategy position
0,2010-07-01,3M CO,0.201311,0.089715
1,2010-07-01,ADOBE SYSTEMS INC,0.053777,0.023651
2,2010-07-01,AMERICAN FINANCIAL GROUP INC,0.054507,0.049387
3,2010-07-01,AMETEK INC,0.038556,0.017183
4,2010-07-01,ANSYS INC,0.535144,0.235356


In [5]:
import Quandl as qd
from utils import david_authtoken

In [6]:
qd_codes = ['YAHOO/'+ticker for ticker in tickers]

In [7]:
if False:
    yahoo_data = qd.get(qd_codes, authtoken=david_authtoken)

In [8]:
dd_path = 'data/ff_yahoo_downloaded_data.csv'
if False:
    yahoo_data.to_csv(dd_path)
else:
    yahoo_data = read_csv(dd_path)

In [9]:
temp_data = yahoo_data.set_index('Date', drop=True)
cols_to_keep = [col for col in temp_data.columns if col.split(' - ')[1] == 'Adjusted Close' ]
temp_data = temp_data[cols_to_keep]
temp_data.columns = [col.split(' - ')[0].split('.')[1] for col in temp_data.columns]
temp_data.columns = [ticker_name.loc[ticker_name['Ticker']==col]['Name'].values[0] for col in temp_data.columns]
temp_data = temp_data.set_index(temp_data.index.to_datetime())
temp_data = temp_data.loc[temp_data.index >= '2010-07-01']

In [10]:
temp_data = temp_data.pct_change().iloc[1:]

In [11]:
data_list = [([col, dat], temp_data[col][dat]) for col in temp_data.columns for dat in temp_data.index]

In [12]:
idx, dta = zip(*data_list)

In [13]:
idx = list(idx)
dta = list(dta)

In [14]:
df = DataFrame(dta)

In [15]:
sec, date = zip(*idx)

In [16]:
df['Security'] = sec

In [17]:
df['Date'] = date

In [18]:
df = df.rename(columns={0: 'Percent Change'})

In [19]:
positions['Date'] = to_datetime(positions['Date'])

In [20]:
positions.drop('Predicted Z-score for transformed percent change', axis=1, inplace=True)

In [21]:
positions.head()

,Date,Security,Strategy position
0,2010-07-01,3M CO,0.089715
1,2010-07-01,ADOBE SYSTEMS INC,0.023651
2,2010-07-01,AMERICAN FINANCIAL GROUP INC,0.049387
3,2010-07-01,AMETEK INC,0.017183
4,2010-07-01,ANSYS INC,0.235356


In [22]:
df = df[['Date', 'Security', 'Percent Change']]

In [23]:
import datetime as dt
def add_year_less_day(s):
    max_date = s['Date'].max()
    n_max_date = dt.datetime(day=max_date.day, month=max_date.month, year=max_date.year+1)
    n_max_date -= timedelta(days=1)
    row = s.loc[s['Date']==max_date]
    pos = row['Strategy position'].values[0]
    sec = row['Security'].values[0]
    return(concat([s, DataFrame([{"Date": n_max_date, "Security": sec, "Strategy position": pos}])]).reset_index(drop=True))
    

In [24]:
positions = positions.groupby('Security').apply(lambda s: add_year_less_day(s))

In [25]:
positions = positions.groupby('Security').apply(lambda df: df.set_index('Date').resample('D').pad().reset_index()).reset_index(drop=True)

In [26]:
pos_pcts = merge(df, positions, on=['Date', 'Security'])

In [27]:
import numpy as np

In [28]:
np.sort(list(set(positions['Date'])))

array([Timestamp('2010-07-01 00:00:00'), Timestamp('2010-07-02 00:00:00'),
       Timestamp('2010-07-03 00:00:00'), ...,
       Timestamp('2016-06-28 00:00:00'), Timestamp('2016-06-29 00:00:00'),
       Timestamp('2016-06-30 00:00:00')], dtype=object)

In [29]:
grouped = pos_pcts.groupby('Date')

In [30]:
def comp_day_ret(info):
    total = (info['Strategy position'] * info['Percent Change']).sum()
    total /= info['Strategy position'].sum()
    return(total)

In [31]:
rets = grouped.apply(comp_day_ret)

In [33]:
cum_rets = (rets+1).cumprod()

In [88]:
ff_factors = read_csv('data/fama_french_5_factors.csv')

In [89]:
ff_factors = ff_factors.rename(columns={'Unnamed: 0': 'Date'}).set_index('Date')

In [85]:
#Here down needs to be fixed
ff_factors.index = ff_factors.index - 19000000

In [87]:
[str(dt.datetime.fromordinal(val)) for val in ff_factors.index.values]

['2467-01-14 00:00:00',
 '2467-01-15 00:00:00',
 '2467-01-16 00:00:00',
 '2467-01-17 00:00:00',
 '2467-01-18 00:00:00',
 '2467-01-21 00:00:00',
 '2467-01-22 00:00:00',
 '2467-01-23 00:00:00',
 '2467-01-24 00:00:00',
 '2467-01-25 00:00:00',
 '2467-01-28 00:00:00',
 '2467-01-29 00:00:00',
 '2467-01-30 00:00:00',
 '2467-01-31 00:00:00',
 '2467-02-01 00:00:00',
 '2467-02-04 00:00:00',
 '2467-02-05 00:00:00',
 '2467-02-06 00:00:00',
 '2467-02-07 00:00:00',
 '2467-02-08 00:00:00',
 '2467-02-11 00:00:00',
 '2467-02-12 00:00:00',
 '2467-04-23 00:00:00',
 '2467-04-24 00:00:00',
 '2467-04-25 00:00:00',
 '2467-04-28 00:00:00',
 '2467-04-29 00:00:00',
 '2467-04-30 00:00:00',
 '2467-05-01 00:00:00',
 '2467-05-02 00:00:00',
 '2467-05-05 00:00:00',
 '2467-05-06 00:00:00',
 '2467-05-07 00:00:00',
 '2467-05-08 00:00:00',
 '2467-05-09 00:00:00',
 '2467-05-12 00:00:00',
 '2467-05-13 00:00:00',
 '2467-05-14 00:00:00',
 '2467-05-15 00:00:00',
 '2467-05-16 00:00:00',
 '2467-05-19 00:00:00',
 '2467-05-20 00: